In [1]:
%reset -f

In [2]:
#.\.venv\Scripts\Activate.ps1

import pandas as pd
import numpy as np
from pathlib import Path

bases_path = Path("bases")

checklist = bases_path / "HERING_FRANQUIA_checklist_RA_RA_FRANQUIA_2025-12-17.xlsx"
relatorio = bases_path / "relatorio-full-lw.xlsx"
estoque = bases_path / "stk17.xlsx"
carteira = bases_path / "CA_aps.xlsx"
carteira_fat = bases_path / "CA_FAT17-12.xlsx"
df_check = pd.read_excel(checklist, usecols=['FILIAL', 'ARTIGO_COR', 'PRESENTE', 'TRANSITO', 'EMBALADO', 'RESERVADO'], engine="calamine", sheet_name="CHECKLIST")
df_relatorio = pd.read_excel(relatorio, header=5, usecols=['Cod. Loja', 'Artigo', 'Cor', 'Atendido (SAP)', 'Cancelada'], engine="calamine")
df_estoque = pd.read_excel(estoque, engine="calamine", usecols=['Ponto Venda Cód', 'Artigo', 'Peças', 'Estoque Total'], sheet_name="stk")
df_cat = pd.read_excel(carteira, engine="calamine", usecols=['PontoVda', 'Sit', 'Artigo', 'Pecas'], sheet_name="Table1")
df_cat_fat_hoje_ontem = pd.read_excel(carteira_fat, engine="calamine", usecols=['PontoVda', 'Artigo', 'Sit', 'Pecas', 'DtEmiNota'], sheet_name="Table1")

In [3]:
df_check.columns

Index(['FILIAL', 'ARTIGO_COR', 'PRESENTE', 'TRANSITO', 'EMBALADO',
       'RESERVADO'],
      dtype='object')

In [4]:
df_check.dtypes

FILIAL         int64
ARTIGO_COR    object
PRESENTE       int64
TRANSITO       int64
EMBALADO       int64
RESERVADO      int64
dtype: object

In [5]:
df_check['ARTIGO_COR'] = df_check['ARTIGO_COR'].str.replace('-', '')
df_check['FILIAL'] = df_check['FILIAL'].astype(str)
df_check['chave'] =  df_check['FILIAL'] + df_check['ARTIGO_COR'] 

In [6]:
df_check.head()

,FILIAL,ARTIGO_COR,PRESENTE,TRANSITO,EMBALADO,RESERVADO,chave
0,10,0111AX7EN,6,0,0,0,100111AX7EN
1,521,77LU1AEN,4,0,0,0,52177LU1AEN
2,521,77LU1AEN,2,0,0,0,52177LU1AEN
3,521,77LU1AEN,2,0,0,0,52177LU1AEN
4,521,77LTN10EN,1,0,0,0,52177LTN10EN


In [7]:
df_estoque.head()

,Ponto Venda Cód,Artigo,Peças,Estoque Total
0,10,0111AX7EN,0,34
1,10,0111M2H07S,1,34
2,10,0111MD307S,0,31
3,10,0111N0A00S,0,159
4,10,0111N1007S,0,67


In [8]:
df_estoque.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75734 entries, 0 to 75733
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Ponto Venda Cód  75734 non-null  int64 
 1   Artigo           75734 non-null  object
 2   Peças            75734 non-null  int64 
 3   Estoque Total    75734 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 2.3+ MB


In [9]:
df_estoque.columns

Index(['Ponto Venda Cód', 'Artigo', 'Peças', 'Estoque Total'], dtype='object')

In [10]:
df_estoque.columns = df_estoque.columns.str.strip()
colunas_numericas = ['Peças', 'Estoque Total']
colunas_categoricas = ['Ponto Venda Cód']
df_estoque[colunas_numericas] = df_estoque[colunas_numericas].astype('Int64')
df_estoque[colunas_categoricas] = df_estoque[colunas_categoricas].astype('str')
print("\n\n---- Depois da limpeza ---")
df_estoque.dtypes



---- Depois da limpeza ---


Ponto Venda Cód    object
Artigo             object
Peças               Int64
Estoque Total       Int64
dtype: object

In [11]:
df_estoque.columns

Index(['Ponto Venda Cód', 'Artigo', 'Peças', 'Estoque Total'], dtype='object')

In [12]:
df_estoque['chave'] =  df_estoque['Ponto Venda Cód'] + df_estoque['Artigo'] 

In [13]:
df_estoque.head()

,Ponto Venda Cód,Artigo,Peças,Estoque Total,chave
0,10,0111AX7EN,0,34,100111AX7EN
1,10,0111M2H07S,1,34,100111M2H07S
2,10,0111MD307S,0,31,100111MD307S
3,10,0111N0A00S,0,159,100111N0A00S
4,10,0111N1007S,0,67,100111N1007S


In [14]:
df_estoque['STK_REAL'] = df_estoque['Estoque Total'] - df_estoque['Peças']

In [15]:
df_estoque.head()

,Ponto Venda Cód,Artigo,Peças,Estoque Total,chave,STK_REAL
0,10,0111AX7EN,0,34,100111AX7EN,34
1,10,0111M2H07S,1,34,100111M2H07S,33
2,10,0111MD307S,0,31,100111MD307S,31
3,10,0111N0A00S,0,159,100111N0A00S,159
4,10,0111N1007S,0,67,100111N1007S,67


In [16]:
df_estoque_agrupado = df_estoque.groupby('chave')['STK_REAL'].sum().reset_index()
df_estoque_agrupado
total = df_estoque_agrupado['STK_REAL'].sum()

print("--- Estoque Agrupado por Chave ---")
print(df_estoque_agrupado)
print(f"\n O total de estoque real no BI é: {total}")

--- Estoque Agrupado por Chave ---
               chave  STK_REAL
0       1000111AX7EN        13
1      1000111M2H07S        10
2      1000111MD307S        13
3      1000111N0A00S        51
4      1000111N1007S        44
...              ...       ...
75729    9N3A7M2H07S        56
75730     9N3A7MD3EN        39
75731    9N3A7N0A00S        71
75732    9N3A7N1007S        70
75733     9N3A7NATEN        33

[75734 rows x 2 columns]

 O total de estoque real no BI é: 2177741


In [17]:
TD_STK =  df_check.groupby(['chave','FILIAL', 'ARTIGO_COR'])['PRESENTE'].sum().reset_index()
TD_STK = TD_STK.merge(df_estoque[['chave', 'STK_REAL']], how='left', on='chave' )
TD_STK['STK_REAL'] = TD_STK['STK_REAL'].fillna(0).astype(np.int64)
TD_STK['DIF_ABS'] = (TD_STK['PRESENTE'] - TD_STK['STK_REAL']).abs()
TD_STK

,chave,FILIAL,ARTIGO_COR,PRESENTE,STK_REAL,DIF_ABS
0,1000111AX7EN,100,0111AX7EN,13,13,0
1,1000111M2H07S,100,0111M2H07S,10,10,0
2,1000111MD307S,100,0111MD307S,13,13,0
3,1000111N0A00S,100,0111N0A00S,51,51,0
4,1000111N1007S,100,0111N1007S,44,44,0
...,...,...,...,...,...,...
64492,9N3A7M2H07S,9,N3A7M2H07S,56,56,0
64493,9N3A7MD3EN,9,N3A7MD3EN,39,39,0
64494,9N3A7N0A00S,9,N3A7N0A00S,71,71,0
64495,9N3A7N1007S,9,N3A7N1007S,70,70,0


In [18]:
TD_STK = TD_STK.groupby('FILIAL')[['PRESENTE', 'STK_REAL', 'DIF_ABS']].sum().reset_index()
display(TD_STK)

,FILIAL,PRESENTE,STK_REAL,DIF_ABS
0,10,5186,5171,41
1,100,3991,3935,78
2,1000,1059,1087,36
3,1004,2334,2322,24
4,1005,6487,6472,15
...,...,...,...,...
523,993,5902,5887,29
524,994,4833,5009,232
525,996,7729,7728,23
526,997,989,983,10


In [19]:
TD_STK['Porcentagem%'] = (TD_STK['DIF_ABS'] / TD_STK['PRESENTE'])*100
TD_STK['Porcentagem%'] = TD_STK['Porcentagem%'].replace([np.inf, -np.inf], np.nan)
TD_STK['Porcentagem%'] = TD_STK['Porcentagem%'].fillna(0).round(0).astype(int)



In [20]:
TD_STK

,FILIAL,PRESENTE,STK_REAL,DIF_ABS,Porcentagem%
0,10,5186,5171,41,1
1,100,3991,3935,78,2
2,1000,1059,1087,36,3
3,1004,2334,2322,24,1
4,1005,6487,6472,15,0
...,...,...,...,...,...
523,993,5902,5887,29,0
524,994,4833,5009,232,5
525,996,7729,7728,23,0
526,997,989,983,10,1


In [21]:
condicoes = [
    TD_STK["DIF_ABS"] == 0,
    TD_STK["Porcentagem%"].between(0,2, inclusive="both"),
    TD_STK["Porcentagem%"].between(3,5, inclusive="both"),
    TD_STK["Porcentagem%"].between(6,9, inclusive="both"),
    TD_STK["Porcentagem%"] >= 10,


]

liberado_choices = ['OK','OK','OK', 'OK', 'NÂO']
obs_choices = ['Sem Diferença', 'Até 2%', 'Até 5%', 'Abaixo de 10%', 'Analisar']

TD_STK['Liberado'] = np.select(condicoes, liberado_choices, default='Revisar Regra')
TD_STK['Obs'] = np.select(condicoes, obs_choices, default='Revisar Regra')

TD_STK['Obs2'] = ''

TD_STK





,FILIAL,PRESENTE,STK_REAL,DIF_ABS,Porcentagem%,Liberado,Obs,Obs2
0,10,5186,5171,41,1,OK,Até 2%,
1,100,3991,3935,78,2,OK,Até 2%,
2,1000,1059,1087,36,3,OK,Até 5%,
3,1004,2334,2322,24,1,OK,Até 2%,
4,1005,6487,6472,15,0,OK,Até 2%,
...,...,...,...,...,...,...,...,...
523,993,5902,5887,29,0,OK,Até 2%,
524,994,4833,5009,232,5,OK,Até 5%,
525,996,7729,7728,23,0,OK,Até 2%,
526,997,989,983,10,1,OK,Até 2%,


In [22]:
print("Iniciando a etapa 2: Reclassificação dos grupos 'Analisar'")

filiais_para_analisar = TD_STK.loc[TD_STK['Obs'] == 'Analisar', 'FILIAL'].unique()
filiais_para_analisar

Iniciando a etapa 2: Reclassificação dos grupos 'Analisar'


array(['1162', '1209', '1284', '331', '346', '998'], dtype=object)

In [23]:
TD_STK_analisar =  df_check.groupby(['chave','FILIAL', 'ARTIGO_COR'])['PRESENTE'].sum().reset_index()
TD_STK_analisar = TD_STK_analisar.merge(df_estoque[['chave', 'STK_REAL']], how='left', on='chave' )
TD_STK_analisar['STK_REAL'] = TD_STK_analisar['STK_REAL'].fillna(0).astype(np.int64)
TD_STK_analisar['DIF_ABS'] = (TD_STK_analisar['PRESENTE'] - TD_STK_analisar['STK_REAL']).abs()
TD_STK_analisar = TD_STK_analisar[TD_STK_analisar['FILIAL'].isin(filiais_para_analisar)]


TD_STK_analisar = TD_STK_analisar[TD_STK_analisar['DIF_ABS'] > 0]
TD_STK_analisar

,chave,FILIAL,ARTIGO_COR,PRESENTE,STK_REAL,DIF_ABS
8567,11620201AX7EN,1162,0201AX7EN,30,40,10
8568,11620201HMJEN,1162,0201HMJEN,13,18,5
8569,11620201M2H07S,1162,0201M2H07S,35,37,2
8570,11620201MD307S,1162,0201MD307S,49,51,2
8571,11620201N0A00S,1162,0201N0A00S,32,65,33
...,...,...,...,...,...,...
64394,998KTNUN10SI,998,KTNUN10SI,60,52,8
64417,998N1TEM2H07S,998,N1TEM2H07S,47,28,19
64418,998N1TEN0A00S,998,N1TEN0A00S,119,85,34
64419,998N1TEN1007S,998,N1TEN1007S,67,48,19


In [24]:
TD_STK_analisar.describe()

,PRESENTE,STK_REAL,DIF_ABS
count,360.000000,360.000000,360.000000
mean,22.430556,28.008333,9.894444
std,22.302570,24.041687,13.495072
min,0.000000,0.000000,1.000000
25%,10.000000,13.000000,2.000000
50%,17.000000,21.500000,6.000000
75%,29.000000,36.000000,11.000000
max,204.000000,171.000000,95.000000


In [25]:
TD_STK_analisar.describe(include='object')

,chave,FILIAL,ARTIGO_COR
count,360,360,360
unique,360,6,138
top,11620201AX7EN,1162,KTNUHMJSI
freq,1,102,6


In [26]:
def reavaliar_analisar(row):
    # Se já foi liberado na primeira etapa, mantém como está
    if row['Obs'] != 'Analisar':
        return row['Liberado'], row['Obs'], row['Obs2']

    filial_atual = row['FILIAL']
    
    # Busca os itens dessa filial no DataFrame detalhado
    # Ajuste 'TD_STK_analisar' se o nome do seu df detalhado for outro
    itens_filial = TD_STK_analisar[TD_STK_analisar['FILIAL'] == filial_atual]
    
    # Se não achar detalhes, retorna o que já estava
    if itens_filial.empty:
        return row['Liberado'], row['Obs'], "Sem detalhes encontrados"

    # Pega os 2 artigos com maior DIF_ABS
    top2_itens = itens_filial.sort_values(by='DIF_ABS', ascending=False).head(2)
    
    # Soma das diferenças dos 2 maiores
    soma_dif_top2 = top2_itens['DIF_ABS'].sum()
    artigos_ids = top2_itens['ARTIGO_COR'].tolist() # Para mensagem
    
    # Cálculo da NOVA Porcentagem (subtraindo os 2 maiores da diferença total)
    # Fórmula: (DIF_TOTAL - DIF_TOP2) / PRESENTE_TOTAL
    nova_dif = row['DIF_ABS'] - soma_dif_top2
    
    # Evita divisão por zero
    novo_perc = (nova_dif / row['PRESENTE'] * 100) if row['PRESENTE'] > 0 else 100
    novo_perc = round(novo_perc, 0) # Arredonda para facilitar comparação

    # --- LÓGICA DE DECISÃO ---
    
    # 1. Se baixou para 9% ou menos
    if novo_perc <= 9:
        return 'OK', 'Analisar', f"Liberar, exceto os artigos-cores {artigos_ids}"
    
    # 2. Se Presente > Stk Real E nova porcentagem entre 10% e 14%
    elif (row['PRESENTE'] > row['STK_REAL']) and (10 <= novo_perc <= 14):
        return 'OK', 'Analisar', f"Liberar, exceto os dois artigos cores, presente > stk BI {artigos_ids}"
    
    
    elif (row['PRESENTE'] < row['STK_REAL']):
        return 'NÃO', 'Analisar', "Muita diferença em vários artigos Presente < Stk BI"
    
    # 4. Caso residual (entre 10-14 mas Presente < Stk, ou entre 14-16, etc)
    else:
        return 'NÃO', 'Analisar', "Muita diferença em vários artigos"

# Aplica a função linha a linha (Apply)
# O zip agrupa os resultados para podermos atribuir a 3 colunas de uma vez
#Juntando as partes: TD_STK.apply(lambda row: ..., axis=1) significa:
#"Para cada linha (que chamaremos de row) no DataFrame TD_STK, execute o código que vem a seguir".
TD_STK[['Liberado', 'Obs', 'Obs2']] = TD_STK.apply(
    lambda row: pd.Series(reavaliar_analisar(row)), axis=1
)

# Exibe o resultado final
TD_STK

,FILIAL,PRESENTE,STK_REAL,DIF_ABS,Porcentagem%,Liberado,Obs,Obs2
0,10,5186,5171,41,1,OK,Até 2%,
1,100,3991,3935,78,2,OK,Até 2%,
2,1000,1059,1087,36,3,OK,Até 5%,
3,1004,2334,2322,24,1,OK,Até 2%,
4,1005,6487,6472,15,0,OK,Até 2%,
...,...,...,...,...,...,...,...,...
523,993,5902,5887,29,0,OK,Até 2%,
524,994,4833,5009,232,5,OK,Até 5%,
525,996,7729,7728,23,0,OK,Até 2%,
526,997,989,983,10,1,OK,Até 2%,


In [27]:
# Cria o agrupamento por 'Liberado' e 'Obs' e conta o tamanho (qtd de linhas)
resumo = TD_STK.groupby(['Liberado', 'Obs']).size().reset_index(name='Total')

# (Opcional) Ordena para ficar mais fácil de ler (Do maior para o menor)
resumo = resumo.sort_values(by='Total', ascending=True)

# Exibe o resumo
resumo



,Liberado,Obs,Total
5,OK,Sem Diferença,1
2,OK,Analisar,2
0,NÃO,Analisar,4
1,OK,Abaixo de 10%,11
4,OK,Até 5%,52
3,OK,Até 2%,458


In [28]:

total_filiais = resumo['Total'].sum()
print(f"Total de registros análise estoque {total_filiais}")

Total de registros análise estoque 528


In [29]:
# 1. Primeiro filtramos apenas as linhas onde Liberado é 'NÃO'
df_travado = TD_STK[TD_STK['Liberado'] == 'NÃO']

# 2. Agrupamos por Filial e Obs2, somando a coluna DIF_ABS
# O reset_index(name='Peças') cria a coluna com o nome 'Peças' igual ao seu print
resumo_travados = df_travado.groupby(['FILIAL', 'Obs2'])['DIF_ABS'].sum().reset_index(name='Peças')

# (Opcional) Ordenar do maior para o menor erro total
resumo_travados = resumo_travados.sort_values(by='Peças', ascending=False)

pd.set_option('display.max_colwidth', None)

# Agora exiba o resumo novamente
resumo_travados

,FILIAL,Obs2,Peças
2,331,Muita diferença em vários artigos Presente < Stk BI,1178
3,346,Muita diferença em vários artigos Presente < Stk BI,805
0,1162,Muita diferença em vários artigos Presente < Stk BI,735
1,1284,Muita diferença em vários artigos Presente < Stk BI,306


In [30]:
df_ressalvas = TD_STK[(TD_STK['Liberado'] == 'OK') & (TD_STK['Obs2'] != '')]

# 2. Agrupa por FILIAL e Obs2, somando o DIF_ABS
resumo_ressalvas = df_ressalvas.groupby(['FILIAL', 'Obs2'])['DIF_ABS'].sum().reset_index(name='Peças')

# 3. Ordena do maior Total para o menor (opcional, ajuda a priorizar)
resumo_ressalvas = resumo_ressalvas.sort_values(by='Peças', ascending=False)

# 4. Garante que o texto da Obs2 apareça inteiro
pd.set_option('display.max_colwidth', None)

# Exibe a tabela
resumo_ressalvas

,FILIAL,Obs2,Peças
1,998,"Liberar, exceto os artigos-cores ['K48RN0ASI', 'KG99N0ASI']",417
0,1209,"Liberar, exceto os artigos-cores ['0227N1007S', '0227MD3EN']",121


In [31]:
df_cat

,PontoVda,Sit,Artigo,Pecas
0,1294,ATENDIDO,76WX13AEN,22
1,1294,PENDENTE,KFTJ2ASI,2
2,1294,PENDENTE,77LU1AEN,3
3,1292,ATENDIDO,0201N0A00S,2
4,1292,ATENDIDO,KGA71ASN,8
...,...,...,...,...
33015,1,ATENDIDO,KTNUHMJSI,137
33016,1,ATENDIDO,KTNUN10SI,38
33017,1,ATENDIDO,KTNUNATSI,46
33018,1,ATENDIDO,KTNUNNJSI,36


In [32]:
df_cat.dtypes

PontoVda     int64
Sit         object
Artigo      object
Pecas        int64
dtype: object

In [33]:
df_cat.columns

Index(['PontoVda', 'Sit', 'Artigo', 'Pecas'], dtype='object')

In [34]:
df_cat.columns = df_cat.columns.str.strip()

colunas_numericas = ['Pecas']
colunas_categoricas = ['Artigo', 'PontoVda', 'Sit']

# --- AQUI ESTÁ A CORREÇÃO ---
# Usamos um loop caso você adicione mais colunas numéricas no futuro
for col in colunas_numericas:
    # Correção: to_numeric -> round(0) -> astype('Int64')
    df_cat[col] = pd.to_numeric(df_cat[col], errors='coerce').round(0).astype('Int64') 

# As categóricas continuam igual
df_cat[colunas_categoricas] = df_cat[colunas_categoricas].astype('str')

print("\n\n---- Depois da limpeza ---")
df_cat.dtypes



---- Depois da limpeza ---


PontoVda    object
Sit         object
Artigo      object
Pecas        Int64
dtype: object

In [35]:
df_cat

,PontoVda,Sit,Artigo,Pecas
0,1294,ATENDIDO,76WX13AEN,22
1,1294,PENDENTE,KFTJ2ASI,2
2,1294,PENDENTE,77LU1AEN,3
3,1292,ATENDIDO,0201N0A00S,2
4,1292,ATENDIDO,KGA71ASN,8
...,...,...,...,...
33015,1,ATENDIDO,KTNUHMJSI,137
33016,1,ATENDIDO,KTNUN10SI,38
33017,1,ATENDIDO,KTNUNATSI,46
33018,1,ATENDIDO,KTNUNNJSI,36


In [36]:
df_cat['chave'] = df_cat['PontoVda'] + df_cat['Artigo']
df_cat

,PontoVda,Sit,Artigo,Pecas,chave
0,1294,ATENDIDO,76WX13AEN,22,129476WX13AEN
1,1294,PENDENTE,KFTJ2ASI,2,1294KFTJ2ASI
2,1294,PENDENTE,77LU1AEN,3,129477LU1AEN
3,1292,ATENDIDO,0201N0A00S,2,12920201N0A00S
4,1292,ATENDIDO,KGA71ASN,8,1292KGA71ASN
...,...,...,...,...,...
33015,1,ATENDIDO,KTNUHMJSI,137,1KTNUHMJSI
33016,1,ATENDIDO,KTNUN10SI,38,1KTNUN10SI
33017,1,ATENDIDO,KTNUNATSI,46,1KTNUNATSI
33018,1,ATENDIDO,KTNUNNJSI,36,1KTNUNNJSI


In [37]:
df_cat["DtEmiNota"] = ''
# 1. Remove a coluna do final e guarda numa variável temporária
coluna_para_mover = df_cat.pop('DtEmiNota')

# 2. Insere ela na posição 4 (Logo após 'Usuario')
# A sintaxe é: .insert(posicao_numerica, 'nome_coluna', conteudo)
df_cat.insert(4, 'DtEmiNota', coluna_para_mover)

# Exibe o resultado
df_cat

,PontoVda,Sit,Artigo,Pecas,DtEmiNota,chave
0,1294,ATENDIDO,76WX13AEN,22,,129476WX13AEN
1,1294,PENDENTE,KFTJ2ASI,2,,1294KFTJ2ASI
2,1294,PENDENTE,77LU1AEN,3,,129477LU1AEN
3,1292,ATENDIDO,0201N0A00S,2,,12920201N0A00S
4,1292,ATENDIDO,KGA71ASN,8,,1292KGA71ASN
...,...,...,...,...,...,...
33015,1,ATENDIDO,KTNUHMJSI,137,,1KTNUHMJSI
33016,1,ATENDIDO,KTNUN10SI,38,,1KTNUN10SI
33017,1,ATENDIDO,KTNUNATSI,46,,1KTNUNATSI
33018,1,ATENDIDO,KTNUNNJSI,36,,1KTNUNNJSI


In [38]:
df_cat_fat_hoje_ontem

,PontoVda,Sit,Artigo,DtEmiNota,Pecas
0,1289,FATURADO,036H7QEN,2025-12-16,1
1,1275,FATURADO,0111N1007S,2025-12-16,9
2,1272,FATURADO,036H7QEN,2025-12-16,5
3,1256,FATURADO,0111AX7EN,2025-12-16,4
4,1256,FATURADO,0111M2H07S,2025-12-16,3
...,...,...,...,...,...
1106,24,FATURADO,N3A7AX7EN,2025-12-16,18
1107,24,FATURADO,N3A7MD3EN,2025-12-16,25
1108,24,FATURADO,N3A7N0A00S,2025-12-16,30
1109,24,FATURADO,N3A7NATEN,2025-12-16,27


In [39]:
df_cat_fat_hoje_ontem.columns

Index(['PontoVda', 'Sit', 'Artigo', 'DtEmiNota', 'Pecas'], dtype='object')

In [40]:
df_cat_fat_hoje_ontem.columns = df_cat_fat_hoje_ontem.columns.str.strip()

colunas_numericas = ['Pecas']
colunas_categoricas = ['Artigo', 'DtEmiNota', 'PontoVda', 'Sit']

# --- AQUI ESTÁ A CORREÇÃO ---
# Usamos um loop caso você adicione mais colunas numéricas no futuro
for col in colunas_numericas:
    # Correção: to_numeric -> round(0) -> astype('Int64')
    df_cat_fat_hoje_ontem[col] = pd.to_numeric(df_cat_fat_hoje_ontem[col], errors='coerce').round(0).astype('Int64') 

# As categóricas continuam igual
df_cat_fat_hoje_ontem[colunas_categoricas] = df_cat_fat_hoje_ontem[colunas_categoricas].astype('str')

print("\n\n---- Depois da limpeza ---")
df_cat_fat_hoje_ontem.dtypes



---- Depois da limpeza ---


PontoVda     object
Sit          object
Artigo       object
DtEmiNota    object
Pecas         Int64
dtype: object

In [41]:
df_cat_fat_hoje_ontem['chave'] = df_cat_fat_hoje_ontem['PontoVda'] + df_cat_fat_hoje_ontem['Artigo']
df_cat_fat_hoje_ontem

,PontoVda,Sit,Artigo,DtEmiNota,Pecas,chave
0,1289,FATURADO,036H7QEN,2025-12-16,1,1289036H7QEN
1,1275,FATURADO,0111N1007S,2025-12-16,9,12750111N1007S
2,1272,FATURADO,036H7QEN,2025-12-16,5,1272036H7QEN
3,1256,FATURADO,0111AX7EN,2025-12-16,4,12560111AX7EN
4,1256,FATURADO,0111M2H07S,2025-12-16,3,12560111M2H07S
...,...,...,...,...,...,...
1106,24,FATURADO,N3A7AX7EN,2025-12-16,18,24N3A7AX7EN
1107,24,FATURADO,N3A7MD3EN,2025-12-16,25,24N3A7MD3EN
1108,24,FATURADO,N3A7N0A00S,2025-12-16,30,24N3A7N0A00S
1109,24,FATURADO,N3A7NATEN,2025-12-16,27,24N3A7NATEN


In [42]:
df_cat_fat_hoje_ontem["DtEmiNota"] = ''
# 1. Remove a coluna do final e guarda numa variável temporária
coluna_para_mover = df_cat_fat_hoje_ontem.pop('DtEmiNota')

# 2. Insere ela na posição 1 (Logo após 'Artigo')
# A sintaxe é: .insert(posicao_numerica, 'nome_coluna', conteudo)
df_cat_fat_hoje_ontem.insert(4, 'DtEmiNota', coluna_para_mover)

# Exibe o resultado
df_cat_fat_hoje_ontem

,PontoVda,Sit,Artigo,Pecas,DtEmiNota,chave
0,1289,FATURADO,036H7QEN,1,,1289036H7QEN
1,1275,FATURADO,0111N1007S,9,,12750111N1007S
2,1272,FATURADO,036H7QEN,5,,1272036H7QEN
3,1256,FATURADO,0111AX7EN,4,,12560111AX7EN
4,1256,FATURADO,0111M2H07S,3,,12560111M2H07S
...,...,...,...,...,...,...
1106,24,FATURADO,N3A7AX7EN,18,,24N3A7AX7EN
1107,24,FATURADO,N3A7MD3EN,25,,24N3A7MD3EN
1108,24,FATURADO,N3A7N0A00S,30,,24N3A7N0A00S
1109,24,FATURADO,N3A7NATEN,27,,24N3A7NATEN


In [43]:

df_consolidado = pd.concat([df_cat, df_cat_fat_hoje_ontem], ignore_index=True)

# Exibe o resultado final
df_consolidado

,PontoVda,Sit,Artigo,Pecas,DtEmiNota,chave
0,1294,ATENDIDO,76WX13AEN,22,,129476WX13AEN
1,1294,PENDENTE,KFTJ2ASI,2,,1294KFTJ2ASI
2,1294,PENDENTE,77LU1AEN,3,,129477LU1AEN
3,1292,ATENDIDO,0201N0A00S,2,,12920201N0A00S
4,1292,ATENDIDO,KGA71ASN,8,,1292KGA71ASN
...,...,...,...,...,...,...
34126,24,FATURADO,N3A7AX7EN,18,,24N3A7AX7EN
34127,24,FATURADO,N3A7MD3EN,25,,24N3A7MD3EN
34128,24,FATURADO,N3A7N0A00S,30,,24N3A7N0A00S
34129,24,FATURADO,N3A7NATEN,27,,24N3A7NATEN


In [44]:
df_consolidado['Pecas'].sum()

np.int64(55805)

In [45]:
df_CA_agrupado = df_consolidado.groupby('chave')['Pecas'].sum().reset_index()
df_CA_agrupado
total = df_CA_agrupado['Pecas'].sum()

print("--- Estoque Agrupado por Chave ---")
print(df_CA_agrupado)
print(f"\n O total de estoque real no BI é: {total}")

--- Estoque Agrupado por Chave ---
               chave  Pecas
0       1000201AX7EN      0
1       1000201HMJEN      0
2      1000201M2H07S      0
3      1000201MD307S      0
4      1000201N1007S      0
...              ...    ...
32503     9N3A7AX7EN      0
32504     9N3A7MD3EN      0
32505    9N3A7N0A00S      0
32506    9N3A7N1007S      0
32507     9N3A7NATEN      0

[32508 rows x 2 columns]

 O total de estoque real no BI é: 55805


In [46]:
TD_CA =  df_check.groupby(['chave','FILIAL', 'ARTIGO_COR'])[['RESERVADO', 'EMBALADO', 'TRANSITO']].sum().reset_index()
TD_CA = TD_CA.merge(df_CA_agrupado[['chave', 'Pecas']], how='left', on='chave' )
TD_CA['CA_BI'] = TD_CA['Pecas'].fillna(0).astype(np.int64)
TD_CA['DIF_ABS'] = (TD_CA['RESERVADO'] - TD_CA['CA_BI']).abs()
TD_CA = TD_CA.drop(columns=['Pecas'])
TD_CA

,chave,FILIAL,ARTIGO_COR,RESERVADO,EMBALADO,TRANSITO,CA_BI,DIF_ABS
0,1000111AX7EN,100,0111AX7EN,0,0,8,0,0
1,1000111M2H07S,100,0111M2H07S,0,0,7,0,0
2,1000111MD307S,100,0111MD307S,0,0,13,0,0
3,1000111N0A00S,100,0111N0A00S,0,0,31,0,0
4,1000111N1007S,100,0111N1007S,0,0,2,0,0
...,...,...,...,...,...,...,...,...
64492,9N3A7M2H07S,9,N3A7M2H07S,0,0,15,0,0
64493,9N3A7MD3EN,9,N3A7MD3EN,0,0,13,0,0
64494,9N3A7N0A00S,9,N3A7N0A00S,0,0,0,0,0
64495,9N3A7N1007S,9,N3A7N1007S,0,0,0,0,0


In [47]:
df_relatorio

,Cod. Loja,Artigo,Cor,Cancelada,Atendido (SAP)
0,1004,0111,AX7EN,0,1
1,1004,0111,M2H07S,0,2
2,1004,0111,MD307S,0,1
3,1004,0111,N0A00S,0,7
4,1004,0111,N0A00S,0,7
...,...,...,...,...,...
2375,456,KZHB,1BSN,0,1
2376,456,KZHB,1DSN,0,2
2377,456,KZHB,1DSN,0,1
2378,456,N1TE,N1007S,0,1


In [48]:
df_relatorio.columns

Index(['Cod. Loja', 'Artigo', 'Cor', 'Cancelada', 'Atendido (SAP)'], dtype='object')

In [49]:
df_relatorio.dtypes

Cod. Loja          int64
Artigo            object
Cor               object
Cancelada          int64
Atendido (SAP)     int64
dtype: object

In [50]:
df_relatorio['Cod. Loja'] = df_relatorio['Cod. Loja'].astype(str)

In [51]:
df_relatorio['chave'] = df_relatorio['Cod. Loja'] + df_relatorio['Artigo'] + df_relatorio['Cor']

In [52]:
df_relatorio = df_relatorio.groupby('chave')[['Atendido (SAP)', 'Cancelada']].sum().reset_index()
df_relatorio
total = df_relatorio['Atendido (SAP)'].sum()

print("--- Estoque Agrupado por Chave ---")
print(df_relatorio)
print(f"\n O total de estoque real no BI é: {total}")

--- Estoque Agrupado por Chave ---
               chave  Atendido (SAP)  Cancelada
0      10040111AX7EN               1          0
1     10040111M2H07S               2          0
2     10040111MD307S               1          0
3     10040111N0A00S              23          0
4     10040111N1007S               1          0
...              ...             ...        ...
1052      87KZHA1BSN               0          0
1053    87N1TEN1007S               1          0
1054     87N3A7MD3EN               1          0
1055    87N3A7N0A00S               0          0
1056     87N3A7NATEN               4          0

[1057 rows x 3 columns]

 O total de estoque real no BI é: 5239


In [53]:
TD_CA = TD_CA.merge(df_relatorio[['chave', 'Atendido (SAP)', 'Cancelada']], how='left', on='chave' )
TD_CA['Atendido (SAP)'] = TD_CA['Atendido (SAP)'].fillna(0).astype(np.int64)
TD_CA['Cancelada'] = TD_CA['Cancelada'].fillna(0).astype(np.int64)
TD_CA

,chave,FILIAL,ARTIGO_COR,RESERVADO,EMBALADO,TRANSITO,CA_BI,DIF_ABS,Atendido (SAP),Cancelada
0,1000111AX7EN,100,0111AX7EN,0,0,8,0,0,0,0
1,1000111M2H07S,100,0111M2H07S,0,0,7,0,0,0,0
2,1000111MD307S,100,0111MD307S,0,0,13,0,0,0,0
3,1000111N0A00S,100,0111N0A00S,0,0,31,0,0,0,0
4,1000111N1007S,100,0111N1007S,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
64492,9N3A7M2H07S,9,N3A7M2H07S,0,0,15,0,0,0,0
64493,9N3A7MD3EN,9,N3A7MD3EN,0,0,13,0,0,0,0
64494,9N3A7N0A00S,9,N3A7N0A00S,0,0,0,0,0,0,0
64495,9N3A7N1007S,9,N3A7N1007S,0,0,0,0,0,0,0


In [54]:
TD_CA['RESERVADO'].sum()

np.int64(31276)

In [55]:
TD_CA['CA_BI'].sum()

np.int64(31688)

In [56]:
TD_CA['Atendido (SAP)'].sum()

np.int64(4024)

In [57]:
TD_CA['DIF_ABS'].sum()

np.int64(978)

In [58]:
TD_CA['Cancelada'].sum()

np.int64(62)

In [59]:
condicoes = [
    (TD_CA['RESERVADO'] > 0) & (TD_CA['CA_BI'] > 0),   # 1. Ambos maiores que 0
    (TD_CA['RESERVADO'] == 0) & (TD_CA['CA_BI'] > 0),  # 2. Reservado zerado, CA Qlik positivo
    (TD_CA['RESERVADO'] == 0) & (TD_CA['CA_BI'] == 0), # 3. Ambos zerados
    (TD_CA['RESERVADO'] > 0) & (TD_CA['CA_BI'] == 0)   # 4. Reservado positivo, CA Qlik zerado
]

# 2. Definimos os Cálculos Matemáticos para cada condição acima (na mesma ordem)
escolhas = [
    (TD_CA['RESERVADO'] - TD_CA['CA_BI']).abs(),       # 1. ABS(Reservado - Qlik)
    (TD_CA['CA_BI'] - TD_CA['TRANSITO']).abs(),        # 2. ABS(Qlik - Transito) -> Atenção aqui!
    TD_CA['RESERVADO'] - TD_CA['CA_BI'],               # 3. Reservado - Qlik
    TD_CA['RESERVADO'] - TD_CA['CA_BI']                # 4. Reservado - Qlik
]

# 3. Cria a coluna aplicando as regras
# O 'default=0' é caso alguma linha não caia em nenhuma regra (segurança)
TD_CA['dif_abs_trs'] = np.select(condicoes, escolhas, default=0)

# Exibe o resultado conferindo as colunas envolvidas
TD_CA[['RESERVADO', 'CA_BI', 'TRANSITO', 'dif_abs_trs']].head(10)

,RESERVADO,CA_BI,TRANSITO,dif_abs_trs
0,0,0,8,0
1,0,0,7,0
2,0,0,13,0
3,0,0,31,0
4,0,0,2,0
5,0,0,8,0
6,0,0,7,0
7,0,0,14,0
8,0,0,10,0
9,0,0,10,0


In [60]:
# A condição traduzida do Excel:
# E(J5<=K5; J5=F5)  --->  (Atendido <= Transito) & (Atendido == CA_QLIK)
condicao = (TD_CA['Atendido (SAP)'] <= TD_CA['TRANSITO']) & (TD_CA['Atendido (SAP)'] == TD_CA['CA_BI'])

# Aplica a lógica: np.where(condicao, valor_se_verdadeiro, valor_se_falso)
TD_CA['dif_aten_BI'] = np.where(
    condicao, 
    (TD_CA['Atendido (SAP)'] - TD_CA['CA_BI']).abs(),  # Se verdadeiro
    TD_CA['DIF_ABS']                                     # Senão (pega o valor que já existe na DIF_ABS)
)

# Exibe o resultado para conferência
TD_CA[['Atendido (SAP)', 'TRANSITO', 'CA_BI', 'DIF_ABS', 'dif_aten_BI']].head(10)

,Atendido (SAP),TRANSITO,CA_BI,DIF_ABS,dif_aten_BI
0,0,8,0,0,0
1,0,7,0,0,0
2,0,13,0,0,0
3,0,31,0,0,0
4,0,2,0,0,0
5,0,8,0,0,0
6,0,7,0,0,0
7,0,14,0,0,0
8,0,10,0,0,0
9,0,10,0,0,0


In [61]:
TD_CA 


,chave,FILIAL,ARTIGO_COR,RESERVADO,EMBALADO,TRANSITO,CA_BI,DIF_ABS,Atendido (SAP),Cancelada,dif_abs_trs,dif_aten_BI
0,1000111AX7EN,100,0111AX7EN,0,0,8,0,0,0,0,0,0
1,1000111M2H07S,100,0111M2H07S,0,0,7,0,0,0,0,0,0
2,1000111MD307S,100,0111MD307S,0,0,13,0,0,0,0,0,0
3,1000111N0A00S,100,0111N0A00S,0,0,31,0,0,0,0,0,0
4,1000111N1007S,100,0111N1007S,0,0,2,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
64492,9N3A7M2H07S,9,N3A7M2H07S,0,0,15,0,0,0,0,0,0
64493,9N3A7MD3EN,9,N3A7MD3EN,0,0,13,0,0,0,0,0,0
64494,9N3A7N0A00S,9,N3A7N0A00S,0,0,0,0,0,0,0,0,0
64495,9N3A7N1007S,9,N3A7N1007S,0,0,0,0,0,0,0,0,0


In [62]:
CA_agrupado = TD_CA.groupby('FILIAL')[['RESERVADO', 'CA_BI', 'EMBALADO', 'TRANSITO', 'DIF_ABS', 'Cancelada', 'dif_abs_trs', 'dif_aten_BI']].sum().reset_index()

CA_agrupado

,FILIAL,RESERVADO,CA_BI,EMBALADO,TRANSITO,DIF_ABS,Cancelada,dif_abs_trs,dif_aten_BI
0,10,2,2,0,1093,0,0,0,0
1,100,36,73,19,1224,37,0,36,37
2,1000,0,0,0,493,0,0,0,0
3,1004,14,14,0,618,0,0,0,0
4,1005,38,18,0,1288,20,0,20,0
...,...,...,...,...,...,...,...,...,...
523,993,12,12,0,1299,0,0,0,0
524,994,2,2,0,619,0,0,0,0
525,996,3241,3211,333,2125,30,0,30,0
526,997,0,0,0,15,0,0,0,0


In [63]:

condicoes_base = [
    CA_agrupado['DIF_ABS'] == 0,
    (CA_agrupado['DIF_ABS'] > 0) & (CA_agrupado['DIF_ABS'] < 11),
    (CA_agrupado['DIF_ABS'] >= 11) & (CA_agrupado['DIF_ABS'] <= 59)
]
escolhas_liberar = ['Ok', 'Ok', 'Ok']
escolhas_obs = ['Sem diferença', 'Até 10 peças de diferença', 'Pouca Diferença']

# Cria as colunas iniciais
CA_agrupado['Liberar'] = np.select(condicoes_base, escolhas_liberar, default='Analisar')
CA_agrupado['Obs'] = np.select(condicoes_base, escolhas_obs, default='Verificar Avançado')


print("Mapeando suspensões por filial...")

# 1. Filtra tudo que está suspenso no Qlik (df_consolidado)
df_suspensos = df_consolidado[
    df_consolidado['Sit'].str.startswith('SUSPENSO', na=False)
]

# 2. Cria um CONJUNTO DE PARCEIROS (Filial, Chave)
# Exemplo: O conjunto vai ter dados assim: {('657', '53C21NEN'), ('100', 'XPTO...')}
# Usamos .astype(str) para garantir que 657 (número) vire '657' (texto) e bata a comparação
chaves_suspensas_por_filial = set(
    zip(df_suspensos['PontoVda'].astype(str), df_suspensos['chave'])
)

# --- FUNÇÃO DE REVALIDAÇÃO ---'

def revalidar_carteira(row):
    # Se já foi liberado, passa reto
    if row['Liberar'] == 'Ok':
        return row['Liberar'], row['Obs']
    
    # Guarda a filial atual como texto para usar na comparação
    filial_atual_str = str(row['FILIAL'])
    
    # Pega os detalhes desta filial na tabela TD_CA
    detalhes = TD_CA[TD_CA['FILIAL'] == row['FILIAL']]
    
    if detalhes.empty:
        return 'NÃO', 'Sem detalhes na TD_CA'

    # --- REGRAS ANTERIORES (Mantidas) ---
    top2_itens = detalhes.sort_values(by='DIF_ABS', ascending=False).head(2)
    soma_dif_top2 = top2_itens['DIF_ABS'].sum()
    artigos_ids_top2 = top2_itens['ARTIGO_COR'].tolist()
    dif_restante_abs = row['DIF_ABS'] - soma_dif_top2
    
    if dif_restante_abs <= 59:
        return 'Ok', "Pouca Diferença"
    elif row['dif_abs_trs'] <= 59:
        return 'Ok', 'Pouca diferença (Regra Trânsito)'
    elif row['dif_aten_BI'] <= 59:
        return 'Ok', 'Pouca diferença (Regra Atendido)'

    top_item_aten = detalhes.sort_values(by='dif_aten_BI', ascending=False).head(2)
    soma_item_top2 = top_item_aten['dif_aten_BI'].sum()
    artigos_ids_aten = top_item_aten['ARTIGO_COR'].tolist()
    saldo_restante_aten = row['dif_aten_BI'] - soma_item_top2

    if saldo_restante_aten <= 59:
        return 'Ok', f"Liberar, exceto {artigos_ids_aten}"

    # --- SUA REGRA FINAL AJUSTADA (Cancelados + Suspensão Confirmada) ---
    else:
        # 1. Soma total de cancelados dessa filial
        total_cancelado = detalhes['Cancelada'].sum()
        
        # 2. Identifica quais chaves têm saldo no Qlik (CA_BI > 0)
        # (São esses itens que "sobram" e precisam estar justificados como suspensos)
        itens_com_saldo_qlik = detalhes[detalhes['CA_BI'] > 0]['chave'].tolist()
        
        # 3. VERIFICAÇÃO RIGOROSA:
        # "Para CADA item que tem saldo no Qlik, ele existe na lista de suspensos DESSA FILIAL?"
        todos_itens_estao_suspensos = True
        
        if len(itens_com_saldo_qlik) == 0:
            # Se não tem nenhum item com saldo Qlik, teoricamente não precisa checar suspensão
            todos_itens_estao_suspensos = False 
        else:
            for chave in itens_com_saldo_qlik:
                # O Pulo do Gato: Verifica se o par (Filial, Chave) existe no conjunto
                if (filial_atual_str, chave) not in chaves_suspensas_por_filial:
                    todos_itens_estao_suspensos = False
                    break # Se um falhar, já era, não é "Todos"
        
        # 4. APLICAÇÃO DA DECISÃO
        # Se cobriu o valor com cancelados E tudo que sobrou no Qlik está realmente suspenso nessa loja
        if (total_cancelado >= row['DIF_ABS']) and todos_itens_estao_suspensos:
            return 'NÃO', 'Todos os pedidos foram cancelados LW'
            
        else:
            return 'NÃO', 'Diferença Crítica'

    # --- EXECUÇÃO ---
print("Rodando validação com verificação de Filial+Chave...")

CA_agrupado[['Liberar', 'Obs']] = CA_agrupado.apply(
    lambda row: pd.Series(revalidar_carteira(row)), axis=1
)

# Exibe o resultado
CA_agrupado.head(6)

Mapeando suspensões por filial...
Rodando validação com verificação de Filial+Chave...


,FILIAL,RESERVADO,CA_BI,EMBALADO,TRANSITO,DIF_ABS,Cancelada,dif_abs_trs,dif_aten_BI,Liberar,Obs
0,10,2,2,0,1093,0,0,0,0,Ok,Sem diferença
1,100,36,73,19,1224,37,0,36,37,Ok,Pouca Diferença
2,1000,0,0,0,493,0,0,0,0,Ok,Sem diferença
3,1004,14,14,0,618,0,0,0,0,Ok,Sem diferença
4,1005,38,18,0,1288,20,0,20,0,Ok,Pouca Diferença
5,1006,22,22,0,8,0,0,0,0,Ok,Sem diferença


In [64]:

# Tente assim se o anterior não funcionar
CARLOS = ['100', '331', '346', '1284']
CA_agrupado_filtrado = CA_agrupado[CA_agrupado['FILIAL'].isin(CARLOS)]

# Agora exibe
display(CA_agrupado_filtrado)

,FILIAL,RESERVADO,CA_BI,EMBALADO,TRANSITO,DIF_ABS,Cancelada,dif_abs_trs,dif_aten_BI,Liberar,Obs
1,100,36,73,19,1224,37,0,36,37,Ok,Pouca Diferença
150,1284,47,78,33,107,31,0,31,31,Ok,Pouca Diferença
245,331,38,71,35,106,33,0,32,33,Ok,Pouca Diferença
252,346,17,20,8,105,3,0,3,3,Ok,Até 10 peças de diferença


In [65]:
# Cria o agrupamento por 'Liberado' e 'Obs' e conta o tamanho (qtd de linhas)
resumo = CA_agrupado.groupby(['Liberar', 'Obs']).size().reset_index(name='Peças')

# (Opcional) Ordena para ficar mais fácil de ler (Do maior para o menor)
resumo = resumo.sort_values(by='Peças', ascending=False)

# O til (~) inverte a lógica: traz tudo que NÃO contém "exceto"
resumo_limpo = resumo[~resumo['Obs'].str.contains("exceto", case=False, na=False)]

print(resumo_limpo['Peças'].sum())
# Exibe o resultado
resumo_limpo



528


,Liberar,Obs,Peças
2,Ok,Sem diferença,422
0,Ok,Até 10 peças de diferença,74
1,Ok,Pouca Diferença,32


In [66]:
# 2. Agrupamos e SOMAMOS as colunas numéricas
# ADICIONE O .reset_index() NO FINAL AQUI 👇
CA_OK = CA_agrupado.groupby(['FILIAL', 'Obs'])[['DIF_ABS', 'dif_abs_trs', 'dif_aten_BI']].sum().reset_index()

# Agora sim essa linha vai funcionar, pois 'Obs' voltou a ser uma coluna
resumo_OK = CA_OK[CA_OK['Obs'].str.contains("exceto", case=False, na=False)]

pd.set_option('display.max_rows', None)
resumo_OK

,FILIAL,Obs,DIF_ABS,dif_abs_trs,dif_aten_BI


In [67]:
# 1. Primeiro filtramos apenas as linhas onde Liberado é 'NÃO'
CA_travado = CA_agrupado[CA_agrupado['Liberar'] == 'NÃO']

# 2. Agrupamos por Filial e Obs2, somando a coluna DIF_ABS
# O reset_index(name='Total') cria a coluna com o nome 'Total' igual ao seu print
CA_travado = CA_travado.groupby(['FILIAL', 'Obs'])['DIF_ABS'].sum().reset_index(name='Total')

# (Opcional) Ordenar do maior para o menor erro total
resumo_travados = CA_travado.sort_values(by='Total', ascending=False)

pd.set_option('display.max_colwidth', None)

# Agora exiba o resumo novamente
resumo_travados

,FILIAL,Obs,Total
